# 1. Daten aus CSV-File in Variable speichern

In [ ]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
csv_file_path = 'Daten.csv'

# Read the CSV file and store it in a variable
data = pd.read_csv(csv_file_path)

print(data)

# 2. Daten aufbereiten

In [6]:
import pandas as pd

# Zähle die Anzahl der Zeilen vor der Bereinigung
rows_before = len(data)
cols_before = len(data.columns)
missing_values_before = data.isna().sum()
total_missing_before = missing_values_before.sum()

# Bereinigung der Daten: Entferne Zeilen mit leeren Feldern
data_clean = data.dropna()

# Alternative: Nur Zeilen entfernen, bei denen alle Felder leer sind
# data_clean = data.dropna(how='all')

# Oder: Nur Zeilen entfernen, bei denen bestimmte Spalten leer sind
# data_clean = data.dropna(subset=['wichtige_spalte1', 'wichtige_spalte2'])

# Zähle die Anzahl der Zeilen nach der Bereinigung
rows_after = len(data_clean)
total_missing_after = data_clean.isna().sum().sum()

# Feedback über gelöschte Daten
print(f"Datensatzanalyse vor der Bereinigung:")
print(f"- Anzahl der Zeilen: {rows_before}")
print(f"- Anzahl der Spalten: {cols_before}")
print(f"- Anzahl der fehlenden Werte insgesamt: {total_missing_before}")
print("\nFehlende Werte pro Spalte:")
print(missing_values_before)

print("\n" + "="*50 + "\n")

print(f"Datensatzanalyse nach der Bereinigung:")
print(f"- Anzahl der Zeilen: {rows_after}")
print(f"- Anzahl der gelöschten Zeilen: {rows_before - rows_after}")
print(f"- Prozent der gelöschten Zeilen: {(rows_before - rows_after) / rows_before * 100:.2f}%")
print(f"- Verbleibende fehlende Werte: {total_missing_after}")

# Speichere den bereinigten Datensatz
# data_clean verweist jetzt auf die bereinigte Version des Datensatzes

Datensatzanalyse vor der Bereinigung:
- Anzahl der Zeilen: 1000000
- Anzahl der Spalten: 1
- Anzahl der fehlenden Werte insgesamt: 200448

Fehlende Werte pro Spalte:
TimeStamp;ActualPos;ActualForce;Nahtnummer;LaserPwr;Befehl    200448
dtype: int64


Datensatzanalyse nach der Bereinigung:
- Anzahl der Zeilen: 799552
- Anzahl der gelöschten Zeilen: 200448
- Prozent der gelöschten Zeilen: 20.04%
- Verbleibende fehlende Werte: 0
